### Supervised learning algorithms:

supervised learning model:

- Use FY2015 icd9 diagnosis code long desciption as training data.
- Use "Other Diagnosis" column from Excel as test data

- Import Traning data, Separate input and output by slicing into 2 the arrays: one is input for training, the other is target. 
- Prepare y output target. Use sklearn.preprocessing.LabelEncoder to encode the Target which is ctr of icd9 dx code.
- Prepare X input array. Perform feature extraction by using klearn.feature_extraction.text  CountVectorizer with common "stop words" (ENGLISH_STOP_WORDS) against icd9 dx long desc.
- Create a corpus (bag-of-words with their usage statistics) from the terms that occur answers.
- Enhanced the input variable X with a weight for every term. Use klearn.feature_extraction.text import TfidfVectorizer
- Use class sklearn.svm.SVC as estimator to fit X/y to implements vector classification.

- Prepare X-test data ("Other Diagnosis") the same way as the training datasets, using TfidfVectorizer.
- Predict X_test with the model created above.





### Import Training data


In [1]:
import pandas as pd
import numpy as np
filename = '~/Downloads/icd9dxref.fy15.txt'
df = pd.read_csv(filename, header=None, delimiter='|', dtype=str, nrows=10000)
df.shape
#print(df)

(10000, 7)

In [2]:
#df1 = df[df[1].str.contains("36")]
df1=df
#print(df1)
df1.shape

(10000, 7)

In [3]:
df1=df1.values

### Array Slicing to split input and output variables
So far, so good; creating and indexing arrays looks familiar.
Now we come to array slicing, and this is one feature that causes problems for beginners to Python and NumPy arrays.
Structures like lists and NumPy arrays can be sliced. This means that a subsequence of the structure can be indexed and retrieved.
This is most useful in machine learning when specifying input variables and output variables, or splitting training rows from testing rows.
Slicing is specified using the colon operator ‘:’ with a ‘from‘ and ‘to‘ index before and after the column respectively. The slice extends from the ‘from’ index and ends one item before the ‘to’ index.

In [4]:
# separate array into input and output components
inputx = df1[:,1:3]  #or X = df1, same result
outputy = df1[:,1]
print(inputx) 
print(outputy)

[['0010' 'CHOLERA DUE TO VIBRIO CHOLERAE']
 ['0011' 'CHOLERA DUE TO VIBRIO CHOLERAE EL TOR']
 ['0019' 'CHOLERA, UNSPECIFIED']
 ...
 ['80121'
  'FRACTURE OF BASE OF SKULL, CLOSED, WITH SUBARACHNOID, SUBDURAL, AND EXTRADURAL HEMORRHAGE, WITH NO LOSS OF CONSCIOUSNESS']
 ['80122'
  'FRACTURE OF BASE OF SKULL, CLOSED, WITH SUBARACHNOID, SUBDURAL, AND EXTRADURAL HEMORRHAGE, WITH BRIEF [LESS THAN ONE HOUR] LOSS OF']
 ['80123'
  'FRACTURE OF BASE OF SKULL, CLOSED, WITH SUBARACHNOID, SUBDURAL, AND EXTRADURAL HEMORRHAGE, WITH MODERATE [1-24 HOURS] LOSS OF CONSC']]
['0010' '0011' '0019' ... '80121' '80122' '80123']


### Prepare Output variable y
Use sklearn.preprocessing.LabelEncoder.
also see sklearn.preprocessing.OrdinalEncoder

In [5]:
>>> from sklearn import preprocessing
>>> le = preprocessing.LabelEncoder()
>>> le.fit(outputy)

LabelEncoder()

In [6]:
>>> list(le.classes_)

['0010',
 '0011',
 '0019',
 '0020',
 '0021',
 '0022',
 '0023',
 '0029',
 '0030',
 '0031',
 '00320',
 '00321',
 '00322',
 '00323',
 '00324',
 '00329',
 '0038',
 '0039',
 '0040',
 '0041',
 '0042',
 '0043',
 '0048',
 '0049',
 '0050',
 '0051',
 '0052',
 '0053',
 '0054',
 '00581',
 '00589',
 '0059',
 '0060',
 '0061',
 '0062',
 '0063',
 '0064',
 '0065',
 '0066',
 '0068',
 '0069',
 '0070',
 '0071',
 '0072',
 '0073',
 '0074',
 '0075',
 '0078',
 '0079',
 '00800',
 '00801',
 '00802',
 '00803',
 '00804',
 '00809',
 '0081',
 '0082',
 '0083',
 '00841',
 '00842',
 '00843',
 '00844',
 '00845',
 '00846',
 '00847',
 '00849',
 '0085',
 '00861',
 '00862',
 '00863',
 '00864',
 '00865',
 '00866',
 '00867',
 '00869',
 '0088',
 '0090',
 '0091',
 '0092',
 '0093',
 '01000',
 '01001',
 '01002',
 '01003',
 '01004',
 '01005',
 '01006',
 '01010',
 '01011',
 '01012',
 '01013',
 '01014',
 '01015',
 '01016',
 '01080',
 '01081',
 '01082',
 '01083',
 '01084',
 '01085',
 '01086',
 '01090',
 '01091',
 '01092',
 '01093',


In [7]:
>>> y = le.transform(outputy)
>>> y

array([   0,    1,    2, ..., 9997, 9998, 9999])

In [8]:
>>> list(le.inverse_transform([4, 2, 1]))

['0021', '0019', '0011']

### Prepare Output variable X

In [9]:
>>> from sklearn.feature_extraction.text import CountVectorizer
>>> from sklearn.feature_extraction import stop_words

In [10]:
#>>> print(stop_words.ENGLISH_STOP_WORDS)
#>>> my_stop_words = stop_words.ENGLISH_STOP_WORDS.union(my_words)
#>>> vectorizer = CountVectorizer(analyzer=u'word',max_df=0.95,lowercase=True,stop_words=set(my_stop_words),max_features=15000)


In [11]:
>>> vectorizer = CountVectorizer(stop_words=stop_words.ENGLISH_STOP_WORDS)
>>> vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'few', 'eight', 'keep', 'his', 'amoungst', 'anyone', 'inc', 'serious', 'along', 'ltd', 'we', 'to', 'every', 'herein', 'less', 'become', 'my', 'seem', 'thin', 'thick', 'empty', 'third', 'no', 'into', 'this', 'often', 'via', 'with', 'is', 'from', 'whole', 'well', 'both', 'therein...he', 'becomes', 'everything', 'whereafter', 'etc', 're', 'see', 'one', 'together', 'a', 'sometime'}),
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [12]:
#>>> t,z = data[:,0], data[:,3] 
>>> corpus = inputx[:,1]
>>> X = vectorizer.fit_transform(corpus)
>>> X

<10000x4370 sparse matrix of type '<class 'numpy.int64'>'
	with 46878 stored elements in Compressed Sparse Row format>

In [13]:
>>> vectorizer.get_feature_names()

['000',
 '10',
 '1000',
 '10th',
 '11th',
 '1249',
 '125',
 '1250',
 '12th',
 '1499',
 '1500',
 '1749',
 '1750',
 '19',
 '1999',
 '1st',
 '20',
 '2000',
 '2009',
 '24',
 '249',
 '2499',
 '25',
 '250',
 '2500',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '39',
 '40',
 '49',
 '499',
 '50',
 '500',
 '59',
 '5q',
 '60',
 '69',
 '70',
 '749',
 '750',
 '79',
 '80',
 '89',
 '90',
 '999',
 '9th',
 'abdomen',
 'abdominal',
 'abducens',
 'abnormal',
 'abnormalities',
 'abnormality',
 'abo',
 'abortion',
 'abortus',
 'abrasion',
 'abscess',
 'absence',
 'abuse',
 'academic',
 'acanthamoeba',
 'acanthosis',
 'acariasis',
 'accessory',
 'accidental',
 'accommodation',
 'accommodative',
 'accretions',
 'acetabulum',
 'acetonuria',
 'achalasia',
 'achieved',
 'achilles',
 'achlorhydria',
 'achromatopsia',
 'acid',
 'acidosis',
 'acne',
 'acoustic',
 'acquired',
 'acrocephalosyndactyly',
 'acromegaly',
 'acting',
 'actinic',
 'actinomycotic',
 'active',
 'activ

In [14]:
>>> X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Hence words that were not seen in the training corpus will be completely ignored in future calls to the transform method:

In [15]:
>>> vectorizer.transform(['Something completely dystrophy.']).toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

As tf–idf is very often used for text features, there is also another class called TfidfVectorizer that combines all the options of CountVectorizer and TfidfTransformer in a single model:

In [16]:
>>> from sklearn.feature_extraction.text import TfidfVectorizer
>>> vectorizer_t = TfidfVectorizer(stop_words=stop_words.ENGLISH_STOP_WORDS)
>>> X = vectorizer_t.fit_transform(corpus)
>>> X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
>>> vectorizer_t.transform(['Something completely dystrophy.']).toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

### Learning and predicting

In the case of the digits dataset, the task is to predict, given an image, which digit it represents. We are given samples of each of the 10 possible classes (the digits zero through nine) on which we fit an estimator to be able to predict the classes to which unseen samples belong.
In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T).
An example of an estimator is the class sklearn.svm.SVC, which implements support vector classification. The estimator’s constructor takes as arguments the model’s parameters.
For now, we will consider the estimator as a black box:

In [18]:
# probability=False by default. Turn off to increase processing speed.
>>> from sklearn import svm
>>> clf = svm.SVC(gamma=0.001, C=100.)

In [19]:
>>> clf.fit(X, y)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Now you can predict new values. In this case, you’ll predict using the last image from digits.data. By predicting, you’ll determine the image from the training set that best matches the last image.


In [20]:
X[:10]

<10x4370 sparse matrix of type '<class 'numpy.float64'>'
	with 25 stored elements in Compressed Sparse Row format>

In [21]:
#>>> clf.predict(X[:-1])
>>> clf.predict(X[:10])

array([0, 1, 2, 3, 6, 6, 6, 7, 8, 9])

##### How far is it from the boundary?
Function “decision_function” that computes the signed distance of a point from the boundary. A negative value would indicate class 0 and a positive value would indicate class 1. Also, a value close to 0 would indicate that the point is close to the boundary.


In [22]:
#>>> clf.decision_function(X[:10])

##### Class Probability
$P(class/input) = 1 / (1 + exp(A * f(input) + B))$

In [23]:
#>>> clf.predict_proba(X[:10])

In [24]:
#>>> list(le.inverse_transform(clf.predict(X[:-1])))
>>> list(le.inverse_transform(clf.predict(X[:100])))

['0010',
 '0011',
 '0019',
 '0020',
 '0023',
 '0023',
 '0023',
 '0029',
 '0030',
 '0031',
 '00320',
 '00321',
 '00322',
 '00323',
 '00324',
 '00329',
 '0038',
 '0039',
 '0040',
 '0041',
 '0042',
 '0043',
 '0048',
 '0049',
 '0050',
 '0051',
 '0052',
 '0053',
 '0054',
 '00581',
 '00589',
 '0059',
 '0060',
 '0061',
 '0062',
 '0063',
 '0064',
 '0065',
 '0066',
 '0068',
 '0069',
 '0070',
 '0071',
 '0072',
 '0073',
 '0074',
 '0075',
 '0078',
 '0079',
 '00800',
 '00801',
 '00802',
 '00803',
 '00804',
 '00809',
 '0081',
 '0082',
 '0083',
 '00841',
 '00842',
 '00843',
 '00844',
 '00845',
 '00846',
 '00847',
 '00849',
 '0085',
 '00861',
 '00862',
 '00863',
 '00864',
 '00865',
 '00866',
 '00867',
 '00869',
 '0088',
 '0090',
 '0091',
 '0092',
 '0093',
 '01090',
 '01001',
 '01002',
 '01003',
 '01004',
 '01005',
 '01006',
 '01010',
 '01011',
 '01012',
 '01013',
 '01014',
 '01015',
 '01016',
 '01080',
 '01081',
 '01082',
 '01083',
 '01084',
 '01085']

### Prepare Sample X_test

In [30]:
>>> inputx_test = [
'blind hypotensive eye',
'headache',
'concussion with no loss of consciousness',
'epiretinal membrane',
'subretinal and intraretinal hemorrhage od with choroidal rupture s/p recent globe exploration',
'pdx traumatic vision loss od',
'suspect corneal abrasion due to irritation from contact lense',
'pseudo aphakia',
's/p os 25c ppv/ilm el with icg/ivf/5f6 20% 1/8/09 2/2 post traumatic macular hole',
'pciol od',
'eye sensitivity to light photophobia',
's/p os 256ppv/icm peel+fc6/ivf/sf6 20% macular hole closed',
'eye trauma od',
'macular puckering',
'dry eye syndrome',
'visit for military services physical',
's/p os 256 ppv/ic6/ivt/sf6 25%',
'eye trauma od',
'strabismus non paralytic exotropia monocular od',
'strabismus non paralytic hypertropia od',
'patient education',
'pdx chronic obstructive disease',
'pciol od',
'eye trauma',
'foreign body intraocular unspecified',
'abrasion of cornea',
'pciol od',
'ddx corneal abrasion',
'ddx corneal ulcer',
's/p ppv/ilm peel/mac hole repair/ivt/sf6 stable',
's/p rt os stable rd precation',
'foveal depression os',
'retinal scar od',
'myopia',
'postsurgical state of eye and adnexa',
'old retinal detachment total or subtotal',
'h/o traumatic macular hole os s/p repair.',
'retina scar stable smalll residual erm on oct',
'eye trauma',
'chronic dacryocystitis',
'routine eye exam',
'traction detachment of retina',
'other proliferative retinopathy',
'retinal detachments with retinal defect',
'superficial injury abrasion of left cornea',
'hypotony of eye',
'epiretinal membrane os',
'uveitis os',
'pseudo aphakia os',
'nuclear sclerosis od',
's/p explosion with corneal fb right eye',
'c/d asymmetry os>od',
'macular puckering',
'adhesions of the posterior synechiae of the iris',
'superficial injury abrasion of cornea right eye',
'some conjunctival irritation left eye',
'des ou possibly allergies but the symptoms do not fit allergic conjunctivitis',
'postsurgical exam',
'hypertrophic scar',
'phthisis bulbi of the right eye',
's/p globe explorations os',
'hemorrhagic conjunctival chemosis os',
'retinal tear without detachment left eye.',
'cataract',
'pdx screening exam',
'unspecified retinal detachment',
'pseudo aphakia posterior chamber od',
'eye trauma',
'commotio retinae with macular hole full thickness ~ stage 2 3. h/o retinal tear os',
'h/o floor/medial wall fx s/p repair mild enophthalmos + lll lag',
'recent retinal detachment total or subtotal',
'traction detachment of retina',
'other proliferative retinopathy',
'epiretinal membrane os reproliferation',
'hypotony os',
'uveitis os',
'vitreous membranes and strands',
'pseudo aphakia os',
'nuclear sclerosis od',
'visit for military services physical',
'closed skull fracture of orbital rim',
'retinal tear left eye',
'preglaucoma open angle with cupping of optic disc ou',
'chorioretinal scar far peripheral',
'diabetes mellitus without mention of complication type ii or unspecified type not stated as uncontrolled',
'postsurgical exam',
'ruptured globe od',
'retinal tear left eye',
'facial injury lip',
'facial injury nose',
'facial injury cheek',
'superficial injury abrasion of right cornea',
'dacryocystitis unspecified',
'periorbital pain',
'contusion with intact skin surface eyeball',
'contusion with intact skin surface eyelid',
'scar hypertrophic',
'scar hypertrophic',
'scar conditions and fibrosis of skin',
'other acquired deformity of head'
... ]

In [27]:
inputx_test

['blind hypotensive eye',
 'headache',
 'concussion with no loss of consciousness',
 'epiretinal membrane',
 'subretinal and intraretinal hemorrhage od with choroidal rupture s/p recent globe exploration',
 'pdx traumatic vision loss od',
 'suspect corneal abrasion due to irritation from contact lense',
 'pseudo aphakia',
 's/p os 25c ppv/ilm el with icg/ivf/5f6 20% 1/8/09 2/2 post traumatic macular hole',
 'pciol od',
 'eye sensitivity to light photophobia',
 's/p os 256ppv/icm peel+fc6/ivf/sf6 20% macular hole closed',
 'eye trauma od',
 'macular puckering',
 'dry eye syndrome',
 'visit for military services physical',
 's/p os 256 ppv/ic6/ivt/sf6 25%',
 'eye trauma od',
 'strabismus non paralytic exotropia monocular od',
 'strabismus non paralytic hypertropia od',
 'patient education',
 'pdx chronic obstructive disease',
 'pciol od',
 'eye trauma',
 'foreign body intraocular unspecified',
 'abrasion of cornea',
 'pciol od',
 'ddx corneal abrasion',
 'ddx corneal ulcer',
 's/p ppv/il

In [31]:
>>> X_test = vectorizer_t.transform(inputx_test)
>>> X_test.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
>>> clf.predict(X_test)

array([3695, 9603, 4437, 3781, 5490, 4065, 3822, 3723, 4270, 4060])

In [32]:
>>> list(le.inverse_transform(clf.predict(X_test)))

['36041',
 '7840',
 '80031',
 '3849',
 '36363',
 '36620',
 '52120',
 '37931',
 '33920',
 '80123',
 '36974',
 '36254',
 '36974',
 '36256',
 '7580',
 '30982',
 '76523',
 '36974',
 '37850',
 '37850',
 '80123',
 '49320',
 '80123',
 '36974',
 '36060',
 '52120',
 '80123',
 '52120',
 '37000',
 '37161',
 '37161',
 '7792',
 '36330',
 '3671',
 '3798',
 '36107',
 '36254',
 '37161',
 '36974',
 '37542',
 '36974',
 '36181',
 '36202',
 '36130',
 '52120',
 '36030',
 '3849',
 '09150',
 '37931',
 '36616',
 '37189',
 '52412',
 '36256',
 '36471',
 '52120',
 '37272',
 '37214',
 '2440',
 '42518',
 '36974',
 '36089',
 '37272',
 '3619',
 '3668',
 '7966',
 '3619',
 '37931',
 '36974',
 '36254',
 '37650',
 '36105',
 '36181',
 '36202',
 '3849',
 '36030',
 '09150',
 '37925',
 '37931',
 '36616',
 '30982',
 '37689',
 '36103',
 '37714',
 '36330',
 '25000',
 '2440',
 '36089',
 '36103',
 '7675',
 '7481',
 '7675',
 '52120',
 '37530',
 '33819',
 '74303',
 '37456',
 '42518',
 '42518',
 '7092',
 '73810']